# 1. Business understanding

### Objectives

Predict an order of magnitude of number of retweets of a given tweet posted in California after about 60 minutes. Prediction model should work on a live data stream.

### Success criteria

The project will be considered successful if the resulting model will work with accuracy of 90% or higher. Results should be produced with a delay no bigger than 1 second.

### Project plan

1. Examine the data provided by the Twitter Streaming API.
1. Create a sample subset of the stream of tweets posted in California
1. After approx. 1 hour enhance the subset with the number of retweets
1. Explore the data gathered in the sample
1. Build and evaluate models
1. Create a spark job that will use the best prediction model
1. Deploy the job to the preset spark cluster